In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Transform coordinates from the solar center-of-mass coordinate system to the topocentric coordinate system defined by observer's location

In [5]:
trajectories = np.loadtxt('coordinates_evolution.csv', delimiter = ',')

In [6]:
def R0(coords):
    rot0 = np.array([[0,1,0],
                 [-1,0,0],
                 [0,0,1]])
    return coords @ rot0

def R1(time, coords, xi_0 = 23.439 * np.pi/180, gamma_0 = 52 * np.pi/180, Omega = 2*np.pi/86164.09053):
    output = np.zeros(coords.shape)
    x = np.cos(xi_0) * np.cos(gamma_0 + Omega * time) * coords[:,0] + np.sin(gamma_0 + Omega * time) * coords[:,1]
    y = -np.cos(xi_0) * np.sin(gamma_0 + Omega * time) * coords[:,0] + np.cos(gamma_0 + Omega * time) * coords[:,1]
    z = np.sin(xi_0) * coords[:,0]
    output[:,0] = x
    output[:,1] = y
    output[:,2] = z
    return output

def R2(time, coords, theta, phi, re = 6371071.03):
    output = np.zeros(coords.shape)
    x = np.cos(theta) * np.cos(phi) * coords[:,0] + np.sin(phi) * np.cos(theta) * coords[:,1] - np.sin(theta) * coords[:,2]
    y = - np.sin(phi) * coords[:,0] + np.cos(phi) * coords[:,1]
    z = np.cos(phi) * np.sin(theta) * coords[:,0] + np.sin(phi) * np.sin(theta) * coords[:,1] + np.cos(theta) * coords[:,2] - re
    output[:,0] = x
    output[:,1] = y
    output[:,2] = z
    return output


In [7]:
t = trajectories[:,0]
sun_a = trajectories[:,1:4]
sun_b = trajectories[:,4:7]
earth = trajectories[:,7:10]
moon = trajectories[:,10:13]

In [8]:
sun_a_rot0 = R0(sun_a)
sun_b_rot0 = R0(sun_b)
earth_rot0 = R0(earth)
moon_rot0 = R0(moon)

In [9]:
sun_a_rot1 = sun_a_rot0 - earth_rot0
sun_b_rot1 = sun_b_rot0 - earth_rot0
moon_rot1 = moon_rot0 - earth_rot0

In [10]:
sun_a_rot2 = R1(t, sun_a_rot1)
sun_b_rot2 = R1(t, sun_b_rot1)
moon_rot2 = R1(t, moon_rot1)

# Berkeley topocentric coordinate generation

In [17]:
berkeley_longitude = -122.4219 * np.pi/180
berkeley_latitude = 37.48205 * np.pi/180

latitude = berkeley_latitude
longitude = berkeley_longitude

theta_0 = np.pi/2 - latitude
phi_0 = longitude

sun_a_rot3 = R2(t, sun_a_rot2, theta_0, phi_0)
sun_b_rot3 = R2(t, sun_b_rot2, theta_0, phi_0)
moon_rot3 = R2(t, moon_rot2, theta_0, phi_0)

sun_a_azi = 180 - np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
# sun_a_azi_se = -np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
r_sun_a = np.sqrt(sun_a_rot3[:,0]**2 + sun_a_rot3[:,1]**2)
sun_a_alt = np.arctan2(sun_a_rot3[:,2], r_sun_a) * 180/np.pi

sun_b_azi = 180 - np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
# sun_b_azi_se = -np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
r_sun_b = np.sqrt(sun_b_rot3[:,0]**2 + sun_b_rot3[:,1]**2)
sun_b_alt = np.arctan2(sun_b_rot3[:,2], r_sun_b) * 180/np.pi

moon_azi = 180 - np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
# moon_azi_se = -np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
r_moon = np.sqrt(moon_rot3[:,0]**2 + moon_rot3[:,1]**2)
moon_alt = np.arctan2(moon_rot3[:,2], r_moon) * 180/np.pi

data_save = np.vstack((t, sun_a_azi, sun_a_alt, sun_b_azi, sun_b_alt, moon_azi, moon_alt)).T
header = 'time sun_a_azi sun_a_alt sun_b_azi sun_b_alt moon_azi moon_alt'
fmt = '%d %.4f %.4f %.4f %.4f %.4f %.4f'
np.savetxt('berkeley_azi_alt.csv', data_save, header = header, fmt = fmt)

# data_save = np.vstack((t, sun_a_azi_se, sun_a_alt, sun_b_azi_se, sun_b_alt, moon_azi_se, moon_alt)).T
# header = 'time sun_a_azi sun_a_alt sun_b_azi sun_b_alt moon_azi moon_alt'
# fmt = '%d %.4f %.4f %.4f %.4f %.4f %.4f'
# np.savetxt('berkeley_azi_alt_se.csv', data_save, header = header, fmt = fmt)

# Shanghai topocentric coordinate geneartion

In [18]:
shanghai_longitude = 121.4845 * np.pi/180
shanghai_latitude = 31.79643 * np.pi/180

latitude = shanghai_latitude
longitude = shanghai_longitude

theta_0 = np.pi/2 - latitude
phi_0 = longitude

sun_a_rot3 = R2(t, sun_a_rot2, theta_0, phi_0)
sun_b_rot3 = R2(t, sun_b_rot2, theta_0, phi_0)
moon_rot3 = R2(t, moon_rot2, theta_0, phi_0)

sun_a_azi = 180 - np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
# sun_a_azi_se = -np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
r_sun_a = np.sqrt(sun_a_rot3[:,0]**2 + sun_a_rot3[:,1]**2)
sun_a_alt = np.arctan2(sun_a_rot3[:,2], r_sun_a) * 180/np.pi

sun_b_azi = 180 - np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
# sun_b_azi_se = -np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
r_sun_b = np.sqrt(sun_b_rot3[:,0]**2 + sun_b_rot3[:,1]**2)
sun_b_alt = np.arctan2(sun_b_rot3[:,2], r_sun_b) * 180/np.pi

moon_azi = 180 - np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
# moon_azi_se = -np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
r_moon = np.sqrt(moon_rot3[:,0]**2 + moon_rot3[:,1]**2)
moon_alt = np.arctan2(moon_rot3[:,2], r_moon) * 180/np.pi

data_save = np.vstack((t, sun_a_azi, sun_a_alt, sun_b_azi, sun_b_alt, moon_azi, moon_alt)).T
header = 'time sun_a_azi sun_a_alt sun_b_azi sun_b_alt moon_azi moon_alt'
fmt = '%d %.4f %.4f %.4f %.4f %.4f %.4f'
np.savetxt('shanghai_azi_alt.csv', data_save, header = header, fmt = fmt)

# Sydney topocentric coordinate generation

In [19]:
sydney_longitude = 151.4365 * np.pi/180
sydney_latitude = -33.88912 * np.pi/180

latitude = sydney_latitude
longitude = sydney_longitude

theta_0 = np.pi/2 - latitude
phi_0 = longitude

sun_a_rot3 = R2(t, sun_a_rot2, theta_0, phi_0)
sun_b_rot3 = R2(t, sun_b_rot2, theta_0, phi_0)
moon_rot3 = R2(t, moon_rot2, theta_0, phi_0)

sun_a_azi = 180 - np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
# sun_a_azi_se = -np.arctan2(sun_a_rot3[:,1], sun_a_rot3[:,0]) * 180/np.pi
r_sun_a = np.sqrt(sun_a_rot3[:,0]**2 + sun_a_rot3[:,1]**2)
sun_a_alt = np.arctan2(sun_a_rot3[:,2], r_sun_a) * 180/np.pi

sun_b_azi = 180 - np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
# sun_b_azi_se = -np.arctan2(sun_b_rot3[:,1], sun_b_rot3[:,0]) * 180/np.pi
r_sun_b = np.sqrt(sun_b_rot3[:,0]**2 + sun_b_rot3[:,1]**2)
sun_b_alt = np.arctan2(sun_b_rot3[:,2], r_sun_b) * 180/np.pi

moon_azi = 180 - np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
# moon_azi_se = -np.arctan2(moon_rot3[:,1], moon_rot3[:,0]) * 180/np.pi
r_moon = np.sqrt(moon_rot3[:,0]**2 + moon_rot3[:,1]**2)
moon_alt = np.arctan2(moon_rot3[:,2], r_moon) * 180/np.pi

data_save = np.vstack((t, sun_a_azi, sun_a_alt, sun_b_azi, sun_b_alt, moon_azi, moon_alt)).T
header = 'time sun_a_azi sun_a_alt sun_b_azi sun_b_alt moon_azi moon_alt'
fmt = '%d %.4f %.4f %.4f %.4f %.4f %.4f'
np.savetxt('sydney_azi_alt.csv', data_save, header = header, fmt = fmt)